<a href="https://colab.research.google.com/github/MrAkash03/Wikipedia_Q-A_Bot/blob/main/WIki_Q%26A_Bot_using_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wikipedia RAG Q&A Bot  
**Retrieval‑Augmented Generation** using Hugging Face + Wikipedia API  

---  

## Cell 1 – Install dependencies

In [ ]:
!pip install -q \
    transformers \
    sentence-transformers \
    faiss-cpu \
    wikipedia \
    torch \
    accelerate \
    bitsandbytes \
    langchain \
    langchain-community \
    langchain-huggingface \
    requests==2.32.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.2 MB/s eta 0:00:00


## Cell 2 – Imports & Global settings

In [ ]:
import os
import re
import json
import faiss
import numpy as np
import wikipedia
from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
EMBEDDER_NAME = "sentence-transformers/all-MiniLM-L6-v2"

# Tone handling
def choose_tone(is_factual: bool) -> str:
    return "professional" if is_factual else "friendly"

## Cell 3 – Build the Wikipedia passage index (once per session)

In [ ]:
# ---------- 1. Download Wikipedia passages ----------
print("Downloading Wikipedia passages (top 10k articles)...")
wikipedia.set_lang("en")

# Grab a list of popular pages
pages = wikipedia.random(pages=10000)   # ~10k articles → ~30‑40k passages

# Split each article into overlapping chunks
def chunk_text(text, chunk_size=256, overlap=64):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
        i += chunk_size - overlap
    return chunks

passages = []
titles   = []

print("Chunking articles...")
for title in tqdm(pages, desc="Chunking"):
    try:
        page = wikipedia.page(title)
        chunks = chunk_text(page.content)
        passages.extend(chunks)
        titles.extend([page.title] * len(chunks))
    except Exception:
        continue   # skip disambiguation / missing pages

print(f"Created {len(passages)} passages from {len(set(titles))} articles.")

Chunking articles...


Chunking:   0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.12/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Created 1892 passages from 463 articles.


## Cell 4 – Encode passages with Sentence‑Transformer

In [ ]:
embedder = SentenceTransformer(EMBEDDER_NAME)
print("Encoding passages (this may take a minute)...")
passage_embeddings = embedder.encode(passages, batch_size=64, show_progress_bar=True, convert_to_numpy=True)

# Normalise for cosine similarity
faiss.normalize_L2(passage_embeddings)

# Build FAISS index
dim = passage_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)   # Inner product = cosine after normalisation
index.add(passage_embeddings)
print(f"FAISS index ready with {index.ntotal} vectors.")

Encoding passages (this may take a minute)...


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

FAISS index ready with 1892 vectors.


## Cell 5 – Load the generative LLM (4‑bit quantised for Colab)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype="auto"
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    max_length=2048,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
    return_full_text=False,
    truncation=True
)

Device set to use cuda:0


## Cell 6 – Retrieval + Generation helper

In [ ]:
MAX_CONTEXT_TOKENS = 1500   # safe headroom for TinyLlama (max 2048)

def truncate_context_to_tokens(passages_with_meta, query, max_tokens):
    """Keep passages until total token count ≤ max_tokens."""
    kept = []
    tokens_used = 0

    # Tokenize query once
    query_ids = tokenizer.encode(query, add_special_tokens=False)
    tokens_used += len(query_ids) + 50
    for passage, title, score in passages_with_meta:
        passage_ids = tokenizer.encode(passage, add_special_tokens=False)
        if tokens_used + len(passage_ids) > max_tokens:
            break
        kept.append((passage, title, score))
        tokens_used += len(passage_ids) + 20
    return kept

def build_prompt(query, contexts):
    # Truncate to safe length
    safe_contexts = truncate_context_to_tokens(contexts, query, MAX_CONTEXT_TOKENS)

    # Simple, clean prompt
    context_str = "\n".join([f"{ctx}" for ctx, _, _ in safe_contexts])
    return f"""Use only the following Wikipedia excerpts to answer the question.
If the answer is not present, reply “I don’t know.”

Excerpts:
{context_str}

Question: {query}
Answer:"""

def answer_question(query):
    retrieved = retrieve(query, k=10)                # pull a few more, then truncate
    if retrieved[0][2] < 0.35:                       # low relevance → chit‑chat
        return None, retrieved

    prompt = build_prompt(query, retrieved)
    output = generator(prompt, max_new_tokens=256)[0]["generated_text"].strip()
    return output, retrieved

## Cell 7 – Simple chit‑chat fallback (rule‑based)

In [ ]:
GREETINGS = ["hi", "hello", "hey", "good morning", "good afternoon", "good evening"]
BYE_WORDS = ["bye", "goodbye", "see you", "exit", "quit"]

def is_greeting(msg):
    return any(g in msg.lower() for g in GREETINGS)

def is_bye(msg):
    return any(b in msg.lower() for b in BYE_WORDS)

def simple_chat(msg):
    msg = msg.lower().strip()
    if is_greeting(msg):
        return "Hello! I'm your Wikipedia Q&A bot. Ask me anything, or just chat. 😊"
    if "how are you" in msg:
        return "I'm doing great, thanks! Ready to fetch Wikipedia knowledge for you."
    if "your name" in msg:
        return "I'm WikiRAG, a Retrieval‑Augmented Generation bot built with Hugging Face."
    if "joke" in msg:
        return "Why did the computer go to art school? Because it wanted to learn how to draw a better *byte*! 😄"
    return "That's interesting! Anything specific you'd like to know from Wikipedia?"

## Cell 8 – Main chat loop (run this cell to start!)

In [ ]:
print("\n" + "="*50)
print("   🤖  Wikipedia RAG Q&A Bot  🤖")
print("="*50)
print("Type your question or just chat. Say **bye** to exit.\n")

while True:
    user_input = input("You: ").strip()
    if not user_input:
        continue

    if is_bye(user_input):
        tone = choose_tone(False)
        print("Bot: Goodbye! Feel free to come back anytime. 👋\n")
        break

    # 1. Try factual answer
    answer, retrieved = answer_question(user_input)
    if answer:
        tone = choose_tone(True)
        print(f"Bot ({tone}): {answer}\n")
        # optional: show sources
        sources = set(title for _, title, _ in retrieved)
        print(f"Sources: {', '.join(sorted(sources))}\n")
    else:
        # 2. Fallback to chit‑chat
        chat_reply = simple_chat(user_input)
        tone = choose_tone(False)
        print(f"Bot ({tone}): {chat_reply}\n")


   🤖  Wikipedia RAG Q&A Bot  🤖
Type your question or just chat. Say **bye** to exit.

You: hi
Bot (friendly): Hello! I'm your Wikipedia Q&A bot. Ask me anything, or just chat. 😊

You: what is the capital of india?
Bot (professional): Delhi is the capital of India.

Sources: Green Line (Kolkata Metro), Janu Basan, Java, Maharagama Divisional Secretariat, Surat, Yodha Ela

You: what is LLM?
Bot (friendly): That's interesting! Anything specific you'd like to know from Wikipedia?

You: what is Large Language Model?
Bot (professional): Large Language Model is a type of machine learning model that can be trained to generate large amounts of text or spoken language. It can be used for language translation, question answering, summarization, and other tasks. Large Language Model can generate natural-sounding text or spoken language that is similar to human language. It can also be used to generate machine translation or summarization models.

Sources: Demographics of the Democratic Republic o

## Cell 8 – Save the Model

In [ ]:
# Example save
import pickle, json
np.save("embeddings.npy", passage_embeddings)
with open("passages.pkl", "wb") as f: pickle.dump(passages, f)
with open("titles.pkl", "wb") as f: pickle.dump(titles, f)